# Playground  
Rodrigo Maciel

In [1]:
from QuantConnect import *
from QuantConnect.Data.Market import TradeBar
from QuantConnect.Research import QuantBook
from datetime import datetime
import pandas as pd

# Step 1: Initialize QuantBook
qb = QuantBook()

# Step 2: Select Data
# Specify symbols for the gold market and S&P500 futures
gold_symbol = qb.AddFuture(Futures.Metals.Gold).Symbol
sp500_symbol = qb.AddFuture(Futures.Indices.SP500EMini).Symbol

# Step 3: Acquire Historical Data
# Retrieve historical data from January 1, 2024, to the present day
start_date = datetime(2014, 1, 1)
end_date = datetime.now()

gold_data = qb.History(gold_symbol, start_date, end_date, Resolution.Daily)
sp500_data = qb.History(sp500_symbol, start_date, end_date, Resolution.Daily)

# Verify the data
print(gold_data.head()) # Display the first few rows of gold data
print(sp500_data.head()) # Display the first few rows of S&P500 data

In [2]:
print(gold_data.index)
print(sp500_data.index)


# Exploración de los datos

## Inspección de los nombres de las columnas y los primeros registros


In [4]:
print("Nombres de columnas:", gold_data.columns)
print("Primeros registros de los datos:\n", gold_data.head())


In [5]:

# Comprobación de valores faltantes
print("\nValores faltantes en cada columna:\n", sp500_data.isnull().sum())
print("\nValores faltantes en cada columna:\n", gold_data.isnull().sum())



In [6]:
# Estadísticas descriptivas básicas
print("\nEstadísticas descriptivas básicas gold:\n", gold_data.describe())



In [7]:
print("\nEstadísticas descriptivas básicas sp500:\n", sp500_data.describe())

## Índices
Los índices pueden ser multidimensionales, dandole más profundidad y especificidad en la búsqueda de datos (por ejemplo, índice contract - symbol - date)

In [8]:
print(gold_data.index)


In [9]:
print(gold_data.index.levels)  # Muestra todos los niveles de índice
print(gold_data.index.get_level_values(0))  # Valores únicos en el primer nivel de índice


### Trabajando con datos de índices

**Selección Basada en Índice**: Utiliza los índices para seleccionar datos. En un DataFrame con índices múltiples, puedes seleccionar datos basándote en una combinación de índices.

python
Copy code
```selected_data = df.loc[('2022-01-01', 'AAPL')]```

**Filtrado Usando Índices**: Filtra el DataFrame basado en condiciones aplicadas a los índices.

`filtered_data = df[df.index.get_level_values('expiry') > '2022-01-01']`

**Agrupación y Agregación**: Agrupa los datos basándote en uno o más niveles de índice y luego realiza operaciones de agregación.

`grouped_data = df.groupby(level='symbol').mean()`

**Reajuste de Índices**: Si es necesario, puedes reajustar los índices para facilitar ciertas operaciones.

`df_reset = df.reset_index()`

**Índices como Columnas**: A veces, puede ser útil convertir los índices en columnas regulares, especialmente para ciertas visualizaciones o para preparar los datos para algoritmos de machine learning.

`df_with_index_as_column = df.reset_index()`

**Iteración sobre Índices**: Para ciertos análisis, puedes necesitar iterar sobre los índices.

``` 
for index, row in df.iterrows():
    print(index, row)     
```


**Análisis de Series Temporales**: En el caso de series temporales, los índices de tiempo permiten realizar operaciones específicas de series temporales, como resampling o window functions.

`monthly_data = df.resample('M').mean()`

## Gráficos

In [10]:
# Step 1: Extract the Relevant Date Component
# Resetting the index to make 'time' a column
gold_data_reset = gold_data.reset_index()

# Extracting the 'time' column and setting it as the new index
gold_data_reset.set_index('time', inplace=True)
gold_prices = gold_data_reset['close']

# Step 2: Plot the Data
try:
    plt.figure(figsize=(10, 6))
    plt.plot(gold_prices.index, gold_prices, label='Gold Price', color='gold')
    plt.title('Gold Market Price Trend')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)
    plt.show()
except Exception as e:
    print("An error occurred while plotting:", e)


In [11]:
# Asumiendo que 'gold_prices' es un DataFrame con una columna 'close'
# Asegúrate de que 'gold_prices' tiene un índice de tipo DatetimeIndex
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Supongamos que 'gold_prices' es una Series con los precios de cierre
returns = gold_prices.pct_change()
volatility = returns.rolling(window=30).std()

# Calculamos la media móvil del precio de cierre para suavizar la línea
sma_50 = gold_prices.rolling(window=50).mean()

# Definimos la banda de volatilidad superior e inferior
upper_band = sma_50 + (volatility * 2 * sma_50)
lower_band = sma_50 - (volatility * 2 * sma_50)

plt.figure(figsize=(14, 7))

# Graficamos el precio y la media móvil
plt.plot(gold_prices.index, gold_prices, label='Gold Price', color='gold')
plt.plot(gold_prices.index, sma_50, label='50-day SMA', color='blue')

# Rellenamos la banda de volatilidad
plt.fill_between(gold_prices.index, lower_band, upper_band, color='grey', alpha=0.3, label='Volatility Band')

plt.title('Gold Price with Volatility Bands')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()



## Bins

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

# Suponiendo que 'gold_prices' es una Serie de pandas con los precios de cierre del oro
# Calculamos los retornos diarios
gold_prices['Returns'] = gold_prices.pct_change()

# Función para dibujar el histograma
def plot_histogram(bins):
    plt.figure(figsize=(10, 6))
    plt.hist(gold_prices['Returns'].dropna(), bins=bins, color='gold', edgecolor='black')
    plt.title('Histograma de retornos diarios del oro')
    plt.xlabel('Retornos')
    plt.ylabel('Frecuencia')
    plt.show()

# Tabla de frecuencias
def frequency_table(bins):
    return pd.cut(gold_prices['Returns'], bins=bins).value_counts()

# Interactividad
@interact
def interactive_histogram(bins=IntSlider(min=10, max=100, step=1, value=50)):
    plot_histogram(bins)
    print(frequency_table(bins))


In [ ]:
gold_data.to_csv('gold_data.csv', index=False)